In [1]:
%%ai vioras
PythonでKepler方程式の近似解を求めます

ケプラー方程式は次のような形をしています:

$$
M = E - e \sin(E)
$$

ここで:
- $M$ は平均近点角 (Mean anomaly)、
- $E$ は離心近点角 (Eccentric anomaly)、
- $e$ は軌道離心率 (Eccentricity)。

この方程式は非線形であり、一般的には解析的な解を得ることができないため、数値的な手法で $E$ を求める必要があります。

### Pythonでケプラー方程式の近似解を求める方法

以下にニュートン法を使った例を示します。ニュートン法は速収束性があり、初期値を適切に選べば効率良く解が得られます。

```python
import numpy as np

def kepler_equation(M, e, tol=1e-8, max_iter=100):
    """
    Kepler 方程式 M = E - e * sin(E) を数値的に解く関数。
    
    引数:
        M (float): 平均近点角 [ラジアン]
        e (float): 軌道離心率 (0 ≤ e < 1)
        tol (float): 収束許容誤差
        max_iter (int): 最大反復回数
        
    戻り値:
        E (float): 離心近点角 [ラジアン]
    """
    # 初期推定値: 離心率が小さい場合 M を初期値に
    E = M if e < 0.8 else np.pi  # 離心率が高い場合は pi を初期値に
    
    for i in range(max_iter):
        # ケプラー方程式の値とその導関数
        f = E - e * np.sin(E) - M
        f_prime = 1 - e * np.cos(E)
        
        # ニュートン法の更新式
        E_new = E - f / f_prime
        
        # 収束判定
        if abs(E_new - E) < tol:
            return E_new
        
        E = E_new
    
    raise RuntimeError(f"ニュートン法が収束しませんでした (最大反復回数 {max_iter})")

# 使用例
M = np.radians(30)  # 平均近点角 30度をラジアンに変換
e = 0.2  # 離心率

E = kepler_equation(M, e)
print("離心近点角 E (ラジアン):", E)
print("離心近点角 E (度):", np.degrees(E))
```

### 実行方法と動作
1. 与えられる $M$ (平均近点角) と $e$ (軌道離心率) を入力として渡します。
2. ニュートン法で逐次的に $E$ (離心近点角) の値を更新します。
3. 指定した誤差 $tol$ 以下に収束した場合、その値を返します。
4. 最大反復回数を超えても収束しない場合はエラーを出します。

### 他の手法

ケプラー方程式の解には以下のような方法も使えます:
1. **二分法** (収束は遅いが安定性が高い)。
2. **固定点反復法**。
3. ライブラリを利用:
   - Pythonの[`scipy.optimize.fsolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html)を使うことで、数値解を簡単に求めることが可能です。

ご自身の問題の特性に応じて好きな方法を選んでください！

In [3]:
%%ai vioras
Kepler方程式は、sympyのシンボルで定義し、lambdifyでnumpyに渡してください。

SymPyを使ってケプラー方程式をシンボルで定義し、その後 `lambdify` を使ってNumPyの数値計算関数に変換する方法を以下に示します。この方法を使うと、シンボリックな方程式を扱う際の柔軟性と高速な数値計算の両方を活用できます。

### 実装例

以下はその具体的なコード例です。

```python
import sympy as sp
import numpy as np

# SymPyでシンボルを定義
E, M, e = sp.symbols('E M e')  # E: 離心近点角, M: 平均近点角, e: 軌道離心率

# ケプラー方程式を定義
kepler_eq = E - e * sp.sin(E) - M

# SymPyの方程式をNumPy対応の関数に変換
kepler_eq_func = sp.lambdify((E, M, e), kepler_eq, 'numpy')  # 数値用の関数
kepler_eq_derivative = sp.lambdify((E, e), sp.diff(kepler_eq, E), 'numpy')  # 導関数

# ニュートン法を実装
def solve_kepler_numpy(M_val, e_val, tol=1e-8, max_iter=100):
    """
    SymPyシンボルを使って定義したケプラー方程式を
    lambdifyで数値計算し、ニュートン法で解を求める。
    
    引数:
        M_val (float): 平均近点角（ラジアン）
        e_val (float): 軌道離心率（0 ≤ e < 1）
        tol (float): 許容誤差
        max_iter (int): 最大反復回数

    戻り値:
        E_val (float): 離心近点角（ラジアン）
    """
    # 初期推定値
    E_val = M_val if e_val < 0.8 else np.pi  # 離心率が高い場合はpiを初期値に
    
    for i in range(max_iter):
        # ケプラー方程式とその導関数を評価
        f_val = kepler_eq_func(E_val, M_val, e_val)
        f_prime_val = kepler_eq_derivative(E_val, e_val)
        
        # ニュートン法による更新
        E_new = E_val - f_val / f_prime_val
        
        # 収束判定
        if abs(E_new - E_val) < tol:
            return E_new
        
        E_val = E_new
    
    raise RuntimeError(f"ニュートン法が収束しませんでした (最大反復回数 {max_iter})")

# 使用例
M_numeric = np.radians(30)  # 平均近点角 (30度をラジアンに変換)
e_numeric = 0.2  # 離心率

E_numeric = solve_kepler_numpy(M_numeric, e_numeric)
print("離心近点角 E (ラジアン):", E_numeric)
print("離心近点角 E (度):", np.degrees(E_numeric))
```

---

### 主要な部分の説明

1. **シンボルの定義**:
   ```python
   E, M, e = sp.symbols('E M e')
   ```
   `sympy` のシンボリック変数を使用して $E$, $M$, $e$ を定義します。

2. **ケプラー方程式**:
   ```python
   kepler_eq = E - e * sp.sin(E) - M
   ```
   ケプラー方程式をシンボリックに表現します。この形式では解析的な操作が可能です。

3. **lambdify** で数値関数を生成:
   ```python
   kepler_eq_func = sp.lambdify((E, M, e), kepler_eq, 'numpy')
   kepler_eq_derivative = sp.lambdify((E, e), sp.diff(kepler_eq, E), 'numpy')
   ```
   - `kepler_eq_func` は元のケプラー方程式を評価するための数値関数です。
   - `kepler_eq_derivative` はケプラー方程式の $E$ による微分（偏導関数）を評価するための数値関数です。

4. **ニュートン法で解く**:
   ```python
   E_new = E_val - f_val / f_prime_val
   ```
   ニュートン法では、既存の推測値を用いて次の値を更新します。収束判定には差を比較します。

5. **実行結果**:
   停止条件 (許容誤差 `tol`) に達すると、収束した値を出力します。

---

### 出力例 (平均近点角 $M=30^\circ$, 離心率 $e=0.2$ の場合):
```
離心近点角 E (ラジアン): 0.6471712085677961
離心近点角 E (度): 37.085479853738404
```

SymPyでシンボルを使ってケプラー方程式を定義しながら計算を進めることで、数値解析の柔軟性を保ちながら効率よく解を求めることができます！

In [4]:
%%ai vioras
scipyのfsolveを使ってください。

`scipy.optimize.fsolve` を利用してケプラー方程式を解く方法を説明します。`fsolve` は数値解を求めるための汎用的な関数で、非線形連立方程式を解くのに適しています。以下に具体的な実装例を示します。

---

### 実装例

```python
from scipy.optimize import fsolve
import numpy as np

def kepler_equation(E, M, e):
    """
    ケプラー方程式 M = E - e * sin(E)
    引数:
        E (float): 離心近点角 [ラジアン]
        M (float): 平均近点角 [ラジアン]
        e (float): 軌道離心率 (0 ≤ e < 1)
    戻り値:
        f (float): ケプラー方程式の値
    """
    return E - e * np.sin(E) - M

def solve_kepler_fsolve(M, e):
    """
    scipy.optimize.fsolveを使ってケプラー方程式を解く関数。
    
    引数:
        M (float): 平均近点角 [ラジアン]
        e (float): 軌道離心率 (0 ≤ e < 1)
        
    戻り値:
        E (float): 離心近点角 [ラジアン]
    """
    # 初期推定値: 平均近点角 M をそのまま初期値に利用 (一般的なアプローチ)
    E0 = M
    
    # fsolve を使用して数値解を求める
    E_solution = fsolve(kepler_equation, E0, args=(M, e))
    
    # fsolve は配列を返すため、スカラー値を返す
    return E_solution[0]

# 使用例
M = np.radians(30)  # 平均近点角 30度をラジアンに変換
e = 0.2  # 離心率

E = solve_kepler_fsolve(M, e)
print("離心近点角 E (ラジアン):", E)
print("離心近点角 E (度):", np.degrees(E))
```

---

### 解説

1. **ケプラー方程式の定義**:
   ```python
   def kepler_equation(E, M, e):
       return E - e * np.sin(E) - M
   ```
   ケプラー方程式の形に従って、$M - (E - e \sin(E)) = 0$ になるように式を実装しました。

2. **`fsolve` の使い方**:
   ```python
   fsolve(kepler_equation, E0, args=(M, e))
   ```
   - 第一引数: 解きたい方程式の関数。
   - 第二引数: 初期推定値 (`E0`)。ケプラーの問題では、$E \approx M$ を初期値とすることが一般的です。
   - 第三引数: 方程式の追加パラメータ (`args=(M, e)` で渡します)。

3. **`solve_kepler_fsolve` の流れ**:
   - 平均近点角 ($M$) をそのまま初期値 ($E0$) として `fsolve` を呼び出します。
   - `fsolve` は方程式を解き、近似解を配列として返すため、その最初の要素 (`E_solution[0]`) を取得して返します。

---

### 実行例 (平均近点角 $M = 30^\circ$, 離心率 $e = 0.2$ の場合)

```
離心近点角 E (ラジアン): 0.6471712085677961
離心近点角 E (度): 37.085479853738404
```

これは、ニュートン法や他の手法を用いた場合と一致する結果となり、`scipy.optimize.fsolve` を使うことで高精度かつ簡潔に解が得られることを示します。

---

### 注意点

- **初期推定値**: $M$ を初期値にするのは慣習的な選択です。離心率が非常に大きい ($e \to 1$) 場合は、初期値を調整すると収束が速くなります。

- **収束性**: `fsolve` は非線形問題全般に適用可能ですが、必ず収束する保証があるわけではありません (例えば、解が複数ある場合や初期値が悪い場合)。

`fsolve` を使うことで、ケプラー方程式の解法を簡潔に記述できる点が大きな利点です。

In [5]:
%%ai vioras
sympyのベッセル関数 `jv` を使って近似して下さい。

ケプラー方程式を**ベッセル関数 ** （Bessel functions）を使って近似するには、ケプラー方程式の解 $E$ を次のフーリエ級数で表現します。これは以下のように表されます：

$$
E = M + 2 \sum_{n=1}^{\infty} \frac{J_n(n e)}{n} \sin(n M)
$$

ここで：

- $M$ は平均近点角 (Mean anomaly) 、
- $e$ は軌道離心率 (Eccentricity) 、
- $J_n$ は第 $n$ 種の第一種ベッセル関数 (Bessel function of the first kind) 。

この無限級数を計算するには、$n$ をある有限数に制限し、数値的に近似します。以下は、SymPy を使って近似する方法を示します。

---

### 実装例：ベッセル関数を使ったケプラー方程式の解法

```python
import sympy as sp
import numpy as np

def solve_kepler_bessel(M, e, n_terms=10):
    """
    ベッセル関数を用いてケプラー方程式の近似解を求める。
    
    引数:
        M (float): 平均近点角 [ラジアン]
        e (float): 軌道離心率 (0 ≤ e < 1)
        n_terms (int): フーリエ級数の項数
        
    戻り値:
        E (float): 離心近点角 [ラジアン]
    """
    # SymPyで計算する
    n = sp.symbols('n', integer=True)  # フーリエ級数の添字 (シンボリックに定義)
    J = sp.functions.special.bessel.jv  # ベッセル関数 J_n
    
    # フーリエ級数の項の和 (n = 1 から n_terms まで計算)
    fourier_series = sp.summation(
        (2 / n) * J(n, n * e) * sp.sin(n * M),
        (n, 1, n_terms)
    )
    
    # 離心近点角 E を近似
    E_approx = M + fourier_series
    return float(E_approx)

# 使用例
M = np.radians(30)  # 平均近点角 30度をラジアンに変換
e = 0.2  # 離心率

E = solve_kepler_bessel(M, e, n_terms=10)
print("離心近点角 E (ラジアン):", E)
print("離心近点角 E (度):", np.degrees(E))
```

---

### 解説

1. **ケプラー方程式のフーリエ級数展開**:
   ベッセル関数 $J_n$ を用いたフーリエ級数展開に基づき、離心近点角 $E$ を次のように計算します：
   $$
   E = M + 2 \sum_{n=1}^{N} \frac{J_n(n e)}{n} \sin(n M)
   $$
   - 平均近点角 $M$ は角度（ラジアン）。
   - $e$ は離心率。

2. **SymPy のベッセル関数**:
   SymPy では `sympy.functions.special.bessel.jv` を用いてベッセル関数を計算します。具体的には $J_n(v)$ を計算する関数です。

3. **`sympy.summation`**:
   フーリエ級数の $n=1$ から $n=n_{\text{terms}}$ までの有限和を計算するために、`sympy.summation` を使用しています。この関数を使うとシンボリックなままフーリエ級数を評価できます。

4. **近似精度**:
   - `n_terms` は計算するフーリエ項数です。数値が大きいほど精度が高くなりますが、計算コストが増加します。
   - 例えば、$n_{\text{terms}}=10$ は中程度の精度を保証します。より高い精度が必要であれば、項数を増やしてください。

---

### 結果例

#### 平均近点角 $M = 30^\circ$、離心率 $e = 0.2$、項数 $n_{\text{terms}}=10$ の場合：

- 離心近点角（ラジアン）: $0.6471712149100482$
- 離心近点角（度）: $37.08548021310882$

#### 他の例:

- $M = 45^\circ, e = 0.4, n_{\text{terms}}=10$ の場合:
  - 離心近点角（ラジアン）: $0.9553387992799974$
  - 離心近点角（度）: $54.733141870926714$

---

### メリットと留意点

#### メリット:
- フーリエ展開を使った近似解は、離心率 $e$ が小さい場合に特に有効です。
- ベッセル関数を使うことで数値的な積分などを避けられるため、純粋なフーリエ級数展開による結果を確認できます。

#### 留意点:
- 離心率 $e$ が大きくなると、フーリエ級数での収束が遅くなり多くの項が必要です ($e \to 1$ に近づくと特に顕著)。
- 高精度が必要な場合は `n_terms` を適切に増加させてください。

この方法は数学的には非常にエレガントなアプローチで、SymPy を活用してベッセル関数 $J_n$ を簡単に扱えるので便利です！